First, I was going to generate a sudoku to be solved programmaticaly.  However, the more I read, the more I realized it took a solver to generate a puzzle (or do it by hand).  For more information see: https://www.101computing.net/sudoku-generator-algorithm/.  So instead I am going to find a single puzzle to test the solver as I build it, then after that is compete I can generate new puzzles.  I pulled my test puzzle from an example here:  https://krazydad.com/sudoku/sfiles/KD_Sudoku_EZ_8_v1.pdf.  More often than not, sudoku solvers use a back-tracking algorithm but I am interested in simulated annealing (both back-tracking and simulated annealing may be useful in a current work project).  The inspiration for the simulated annealing comes from https://xianblog.wordpress.com/2010/02/23/sudoku-via-simulated-annealing/, https://www.researchgate.net/publication/220704743_Sudoku_Using_Parallel_Simulated_Annealing, and https://www.youtube.com/watch?v=FyyVbuLZav8.

$$\begin{aligned}
\text{Test Puzzle}
\begin{array}{|ccc|ccc|ccc|}
\hline
2 & & 5 & & & 7 & & & 6 \\
4 & &   & 9 & 6 & & & 2 & \\
  & &  & & 8 & & & 4 & 5 \\
\hline
9 & 8 & & & 7 & 4 & & & \\
5 & 7 & & 8 & & 2 & & 6 & 9 \\
  & & & 6 & 3 & & & 5 & 7 \\
\hline
7 & 5 & & & 2 & & & & \\
  & 6 & & & 5 & 1 & & & 2 \\
3 & & & 4 & & & 5 & & 8 \\
\hline
\end{array}
\end{aligned}$$

Some other interesting tips & tricks may have been implemented from here:  https://towardsdatascience.com/15-tips-and-tricks-for-jupyter-notebook-that-will-ease-your-coding-experience-e469207ac95c

A GUI was developed by looking at the following pages:  http://newcoder.io/gui/part-3/, https://stackoverflow.com/questions/48269694/sudoku-9x9-gui-grid-python-tkinter-buttons, https://www.daniweb.com/programming/software-development/tutorials/520379/how-to-write-a-sudoku-gui-in-python-wxpython, https://trevorappleton.blogspot.com/2013/10/guide-to-creating-sudoku-solver-using.html, http://buklijas.info/blog/2018/10/01/making-web-apps-with-jupyter-notebook/

I ran into similar problems as this article and found it helpful:  https://www.adrian.idv.hk/2019-01-30-simanneal/, also youtu.be/E8tkpzDne7I

# Step 0: Install Packages

In [ ]:
import numpy as np
import random
import math
import statistics
import pandas as pd

from math import exp
from matplotlib import pyplot


# Step 1:  Set Up Initial Puzzle

In [ ]:
grid = np.array(
    [
        [2, 0, 5, 0, 0, 7, 0, 0, 6],
        [4, 0, 0, 9, 6, 0, 0, 2, 0],
        [0, 0, 0, 0, 8, 0, 0, 4, 5],
        [9, 8, 0, 0, 7, 4, 0, 0, 0],
        [5, 7, 0, 8, 0, 2, 0, 6, 9],
        [0, 0, 0, 6, 3, 0, 0, 5, 7],
        [7, 5, 0, 0, 2, 0, 0, 0, 0],
        [0, 6, 0, 0, 5, 1, 0, 0, 2],
        [3, 0, 0, 4, 0, 0, 5, 0, 8],
    ]
)
# print(grid)

testgrid = np.empty_like(grid)
testgrid[:] = grid

nonzerocount = np.count_nonzero(grid)
zerocount = 81 - nonzerocount

gridaddresses = [[-1, -1]]

for i in range(0, 9):
    # print(grid)
    for j in range(0, 9):
        # print(i,j,grid[i, j])
        # print(i,j,grid[0:i+1,j])
        # if testgrid[i][j] != 0:
        #    testgrid[i][j] = 1
        if grid[i, j] == 0:

            # print(grid[i,j])
            if j >= 0:
                # print(grid[i,j])
                checkset = set(grid[i, :])
                gridaddresses = np.vstack([gridaddresses, [i, j]])
                rndnumforgrid = random.randint(1, 9)
                grid[i, j] = rndnumforgrid
                # print(i, j)
                # print(grid[i,0:j])
                # print(i, j, checkset, grid[i, :])
                # print(grid[i][j])
                # print(grid[i][j] in checkset)
                # ******** THE SECTION BELOW ISN'T QUITE RIGHT, FIX IT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!**************
                if len(checkset) <= 9 and 0 in checkset:
                    while grid[i, j] in checkset:
                        # print(checkset, grid[i, :], i, j)
                        # print(grid[i,:])
                        rndnumforgrid = random.randint(1, 9)
                        grid[i, j] = rndnumforgrid

                # print(i, j, checkset, grid[i, :])

# print(grid)
# print(testgrid)

# Step 2:  Calculate Cost Function

In [ ]:
# Define a function to Calculate Cost Function
def CostFunction(ingrid):
    CostFuncVal = 0
    for i in range(0, 9):
        CostFuncVal = CostFuncVal + 9 - len(set(ingrid[i, :]))
        # print("row", i, CostFuncVal, len(set(ingrid[i, :])), ingrid[i, :])
    for j in range(0, 9):
        CostFuncVal = CostFuncVal + 9 - len(set(ingrid[:, j]))
        # print("column",j,CostFuncVal,len(set(ingrid[:, j])),np.transpose(ingrid[:, j]))
    for iii in range(0, 3):
        for jjj in range(0, 3):
            CostFuncVal = CostFuncVal + (
                9 - len(set(ingrid[iii : iii + 3, jjj : jjj + 3].flatten()))
            )
            # print("square",iii,jjj,CostFuncVal,len(set(ingrid[iii : iii + 3, jjj : jjj + 3].flatten())),ingrid[iii : iii + 3, jjj : jjj + 3].flatten(),)
    return CostFuncVal


CostFunction(grid)

In [ ]:
def sim_anneal_iter(input_grid,var_grid,t):  
#input grid is the puzzle to be solved, var_grid is the unit grid showing which positions are variables
    iniCost = CostFunction(input_grid)
    
    randmethod = random.randint(0,2)
    #print(randmethod)
    
    if randmethod == 0:
        randrow = random.randint(0, 8)
        rowzeroidxs = np.nonzero(var_grid[randrow,] == 0)[0]

        idx1 = random.randint(0, len(rowzeroidxs) - 1)
        idx2 = random.randint(0, len(rowzeroidxs) - 1)

        pos1 = rowzeroidxs[idx1]
        pos2 = rowzeroidxs[idx2]

        while pos1 == pos2:
            pos2 = rowzeroidxs[random.randint(0, len(rowzeroidxs) - 1)]

        if pos1 != pos2:
            gridval1 = input_grid[randrow, pos1]
            gridval2 = input_grid[randrow, pos2]
            input_grid[randrow, pos1] = gridval2
            input_grid[randrow, pos2] = gridval1

        checkCost = CostFunction(input_grid)

        diff = checkCost - iniCost
        #print(diff,t)
        # calculate metropolis acceptance criterion
        if diff >= 0:
            metropolis = exp(-diff / t)
    
            #check if we the new point is better, otherwise switch back to initial
            metcheck = random.random()
            if metcheck >= metropolis:
                input_grid[randrow, pos1] = gridval1
                input_grid[randrow, pos2] = gridval2

            if metcheck < metropolis:
                iniCost = checkCost
                
        if diff < 0:
            iniCost = checkCost
        
    elif randmethod == 1:
        randcol = random.randint(0, 8)
        colzeroidxs = np.nonzero(var_grid[randcol,] == 0)[0]

        idx1 = random.randint(0, len(colzeroidxs) - 1)
        idx2 = random.randint(0, len(colzeroidxs) - 1)

        pos1 = colzeroidxs[idx1]
        pos2 = colzeroidxs[idx2]

        while pos1 == pos2:
            pos2 = colzeroidxs[random.randint(0, len(colzeroidxs) - 1)]

        if pos1 != pos2:
            gridval1 = input_grid[pos1, randcol]
            gridval2 = input_grid[pos2, randcol]
            input_grid[pos1, randcol] = gridval2
            input_grid[pos2, randcol] = gridval1

        checkCost = CostFunction(input_grid)

        diff = checkCost - iniCost
        #print(diff,t)
        # calculate metropolis acceptance criterion
        if diff >=0:
            metropolis = exp(-diff / t)
    
            #check if we the new point is better, otherwise switch back to initial
            metcheck = random.random()
            if metcheck >= metropolis:
                input_grid[pos1, randcol] = gridval1
                input_grid[pos2, randcol] = gridval2

            if metcheck < metropolis:
                iniCost = checkCost
        if diff < 0:
            iniCost = checkCost
        
    elif randmethod == 2:
        randsqr1 = random.randint(0,2)
        randsqr2 = random.randint(0,2)
        square = var_grid[(randsqr1)*3:(randsqr1+1)*3,(randsqr2)*3:(randsqr2+1)*3] 
        org_square = input_grid[(randsqr1)*3:(randsqr1+1)*3,(randsqr2)*3:(randsqr2+1)*3]
        changeable = np.array(np.where(org_square == 0)).T
        if len(changeable) < 2:
            return iniCost
        idx1, idx2 = random.sample(list(map(tuple, changeable)), 2)
        row1, col1 = idx1
        row2, col2 = id2
        square[row1,col1], square[row2,col2] = square[row2,col2], square[row1,col1]


    checkCost = CostFunction(input_grid)

    diff = checkCost - iniCost
    
    if diff < 0:
        iniCost = checkCost
        return iniCost
    else:
        metropolis = exp(-diff / t)
    
        #check if we the new point is better, otherwise switch back to initial
        metcheck = random.random()
        if metcheck < metropolis:
            iniCost = checkCost
            return checkCost
        else:
            if randmethod == 0:
                input_grid[randrow,pos1] = gridval1
                input_grid[randrow,pos2] = gridval2
            elif randmethod == 1:
                input_grid[pos1,randcol] = gridval1
                input_grid[pos2,randcol] = gridval2
            else:
                square[row1,col1],square[row2,col2] = square[row2,col2],square[row1,col1]
            return iniCost

In [ ]:
# launch window
# main
# initial temperature
temp = 0.5

# define the total iterations
n_iterations = 250000000
cooling_rate = 0.9999995
tt = temp

# run the algorithm
for i in range(n_iterations):
    # calculate temperature for current epoch
    tt = tt * cooling_rate #(1-1/n_iterations)  #  / (1 + math.log(i + 1) / 10)  #** (math.floor((i + 1) / 10))-/
    # print(tt)

    #    print(CostFunction(grid))
    
    if i % 100000 == 0:
        CostCheck = CostFunction(grid)
        print(tt, CostCheck, i)
    if CostCheck == 0:
        End
    # swap two numbers in solution
    sim_anneal_iter(grid, testgrid, tt)

# print(grid)
# for i in range(0, 9):
# print(i, set(grid[i, :]), grid[i, :])
# for j in range(0, 9):
# print(j, set(grid[:, j]), grid[:, j])
# for iii in range(0, 3):
# for jjj in range(0, 3):
# print(
#    iii,
#    jjj,
#    set(grid[iii : iii + 3, jjj : jjj + 3].flatten()),
#    grid[iii : iii + 3, jjj : jjj + 3].flatten(),
# )

CostFuncVal = 0
for i in range(0, 9):
    CostFuncVal = CostFuncVal + 9 - len(set(grid[i, :]))
    print("row", i, CostFuncVal, len(set(grid[i, :])), grid[i, :])
for j in range(0, 9):
    CostFuncVal = CostFuncVal + 9 - len(set(grid[:, j]))
    print("column", j, CostFuncVal, len(set(grid[:, j])), np.transpose(grid[:, j]))
for iii in range(0, 3):
    for jjj in range(0, 3):
        CostFuncVal = (
            CostFuncVal + 9 - len(set(grid[iii : iii + 3, jjj : jjj + 3].flatten()))
        )
        print(
            "square",
            iii,
            jjj,
            CostFuncVal,
            len(set(grid[iii : iii + 3, jjj : jjj + 3].flatten())),
            grid[iii : iii + 3, jjj : jjj + 3].flatten(),
        )

print(grid)
print(testgrid)

In [ ]:
grid = np.array(
    [
        [2, 0, 5, 0, 0, 7, 0, 0, 6],
        [4, 0, 0, 9, 6, 0, 0, 2, 0],
        [0, 0, 0, 0, 8, 0, 0, 4, 5],
        [9, 8, 0, 0, 7, 4, 0, 0, 0],
        [5, 7, 0, 8, 0, 2, 0, 6, 9],
        [0, 0, 0, 6, 3, 0, 0, 5, 7],
        [7, 5, 0, 0, 2, 0, 0, 0, 0],
        [0, 6, 0, 0, 5, 1, 0, 0, 2],
        [3, 0, 0, 4, 0, 0, 5, 0, 8],
    ]
)

print(np.argwhere(grid == 0))

In [ ]:
print(math.log(2))